<a href="https://colab.research.google.com/github/adong-hood/cs200/blob/main/ch_6_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Reading

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

<ul>
    <li><a href = "https://nextjournal.com/sdanisch/cartographic-visualization">Cartographic Visualization</a></li>
</ul>
    

### This section covers a few large concepts that work together in one big example:
<ul>
    <li>Using a web API to get information</li>
    <li>Applying a function to a data set and getting a new column</li>
    <li>Mapping data using Altair</li>
<ul>

## Preliminaries

In [ ]:
import altair as alt
import pandas as pd
from vega_datasets import data
import requests
# for Jupyter Notebook only. alt.renderers.enable('notebook')

Note: Vega Datasets is a common repository for example datasets used by Vega related projects. Vega is a visualization grammar, a declarative language for creating, saving, and sharing interactive visualization designs. With Vega, you can describe the visual appearance and interactive behavior of a visualization in a JSON format, and generate web-based views using Canvas or SVG.

## Warmup with the book example

Let’s take on the seemingly simple task of plotting some of the country data on a map like we did in Google Sheets earlier.

Altair provides us with the facility to make a blank map. The counties data that is passed to the chart is the data needed to create and outline the map.

### altair.topo_feature
<code>altair.topo_feature(url, feature, **kwargs)</code>
<p>A convenience function for extracting features from a topojson url</p>

Parameters:
url:string
An URL from which to load the data set.

feature:string
The name of the TopoJSON object set to convert to a GeoJSON feature collection. For example, in a map of the States, there may be an object set named “counties”. Using the feature property, we can extract this set and generate a GeoJSON feature object for each county.

**kwargs :
additional keywords passed to TopoDataFormat



In [ ]:
counties = alt.topo_feature(data.us_10m.url, 'counties')

In [ ]:
alt.Chart(counties).mark_geoshape().project(
    type='albersUsa').properties(
    width=500,
    height=300
)

What we want to do is graphing the unemployment data by county.

In [ ]:
unemp_data = data.unemployment.url
unemp_data

In [ ]:
unemp_data = pd.read_csv(unemp_data,sep='\t')
unemp_data.head()

Using the <code>transform_lookup</code> method, we can arrange for the id in the geographic data to be matched against the id in our unemp_data data frame.

In [ ]:
alt.Chart(counties).mark_geoshape(
).encode(
    color='rate:Q'
).transform_lookup(
    lookup='id',
    from_=alt.LookupData(unemp_data, 'id', ['rate'])
).project(
    type='albersUsa'
).properties(
    width=500,
    height=300,
    title='Unemployment by County'
)

## Part 1 - Adding country codes to our data.


Let's try a more complicated case where we do not have all the data readily available to us.This time we will plot average incomes of counties on map.

### Read in country income data

This data set contains a few countries average income, but it does not have each country's numeric code which is needed for transform_lookup when mapping income on map.

Our goal is add unique country numeric code to this set so that we can map them.

In [ ]:
income_data = pd.read_csv('http://pluto.hood.edu/~dong/datasets/country_income.csv')
print(income_data.shape)
income_data.head()

### Look up country code for one country

We need to find countries' numeric code on line.
<p>we import the requests module, then we request what we want, and save the results as res.</p>


In [ ]:
#requst
res = requests.get('https://restcountries.com/v2/alpha/aus')
res.status_code

<p><code>get </code> method does the following - first it goes to the website https://restcountries.com/v2/alpha/usa, then returns the information for that country in json format.</p>
<ul>
<!--li>/rest - technically REST stands for REpresentational State Transfer. This uses the HTTP protocol to ask for and respond with data.</li-->
<li>/v2 - this is version 2 of this website’s protocol</li>
<li>/alpha - This tells the website that the next thing we are going to pass tell it is the three letter code for the country.</li>
<li>/AUS - this can be any valid three letter country code. for example usa</li>
</ul>

In [ ]:
res.text

In [ ]:
temp = res.json()
temp

In [ ]:
temp['numericCode']

In [ ]:
res = requests.get('https://restcountries.com/v2/alpha/bra')
print(res.json()['numericCode'])

## Adding country code to income data

<p>We now implement a function that takes a country letter code and fetch its numeric code from the web using Web API. This will allow us to add numeric code to all countries easily.</p>

In [ ]:
#function
def look_up_code(country_code):
    address = 'https://restcountries.com/v2/alpha/'+country_code
    res = requests.get(address)
    country_info = res.json()
    country_num = country_info['numericCode']
    return int(country_num)

<p><code>map</code> is a method of a Series, so we use the syntax df.myColumn.map(function). This applies the function we pass as a parameter to each element of the series and constructs a brand new series. Add a new column <code>country code</code> with the new series after map.</p>

In [ ]:
income_data.head(2)

In [ ]:
#map and add another column
income_data['countrycode'] = income_data.LOCATION.map(look_up_code)
print(income_data.shape)
income_data.head()


In [ ]:
income_data[income_data['LOCATION'] == 'BRA']

# Part 2 - Mapping Income Data

## 2.1 Getting a Blank Map

In [ ]:
# get the countries objects.
countries = alt.topo_feature(data.world_110m.url, 'countries')
print(type(countries))
#l = data.world_110m.url
#l

The above code indicates that we want to extract Geo features from the specified url for the countries object. All countries are represented using numeric code. Let us draw the world map by passing countries to Altair.

Projections map from a data domain (spatial position) to a visual range (pixel position). We can also specify projection parameters, such as scale (zoom level) and translate (panning), to customize the projection settings.

In [ ]:
#blank map
alt.Chart(countries).mark_geoshape(
    fill='#666666',
    stroke='white'
).properties(
    width=750,
    height=450
).project('equirectangular')

## 2.2 Adding Income Data to the Map

To encode income data in the geo shape, using the <code>transform_lookup</code> method, we can arrange for country in the geographic data to be matched against the country in our income data frame.

In [ ]:
alt.Chart(countries).mark_geoshape(stroke='black', strokeWidth=0.5).encode(
    tooltip = 'LOCATION:N',
    color=alt.Color('Value:Q', scale=alt.Scale(scheme='plasma'))
).transform_lookup(
    lookup='id',
    from_=alt.LookupData(income_data, 'countrycode', ['Value', 'LOCATION'])
).project(
    type = 'equirectangular'
).properties(
    width=750,
    height=450,
    title = "Income by Country"
)

The big white space indicates we only have income data for a small number of countries.

# Your implementation of section 6.3 starts from here...

Clearly mark the question number and the map.